In [6]:
import os
import csv
import pandas as pd
import numpy as np
import ssl
import smtplib
import request
from dotenv import load_dotenv
from tabulate import tabulate
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount

# Load environement variables
load_dotenv()

# Getting Acces ID from env_vars
my_app_id = os.environ.get("MY_APP_ID")
my_app_secret = os.environ.get("MY_APP_SECRET")
my_access_token = os.environ.get("MY_ACCESS_TOKEN")


# Initialitiong connection with Facebook
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

# Connecting to accounts
laSalle_account = AdAccount(os.environ.get("LASALLE_ADACCOUNT"))

# Seting-Up my requests
params = {
    'date_preset': "last_week_mon_sun",
    "level": "campaign",
    'time_increment': "1"
}

fields = ["campaign_name", "reach", "clicks", "spend"]

# Requesting insights
laSalle_insights = laSalle_account.get_insights(params=params, fields=fields)

# Transform insights into a list of dict
laSalle_insights = [dict(x) for x in laSalle_insights]

# Print result
print(laSalle_insights)

[{'id': None, 'campaign_name': 'CL - Campagne permanente - Boost post - PO004623', 'reach': '18251', 'clicks': '2282', 'spend': '172.24', 'date_start': '2019-04-01', 'date_stop': '2019-04-01'}, {'id': None, 'campaign_name': 'eLearning - Conversion - Budget Optimization - PO000000', 'reach': '5898', 'clicks': '73', 'spend': '97.93', 'date_start': '2019-04-01', 'date_stop': '2019-04-01'}, {'id': None, 'campaign_name': 'CLM - Conversion - Discussion DEC Phase 2 - Mars 2019 - FR - PO005153', 'reach': '7588', 'clicks': '61', 'spend': '65.08', 'date_start': '2019-04-01', 'date_stop': '2019-04-01'}, {'id': None, 'campaign_name': 'CLM - Conversion - Discussion DEC Phase 2 - Mars 2019 - EN - PO005153', 'reach': '7744', 'clicks': '35', 'spend': '51.4', 'date_start': '2019-04-01', 'date_stop': '2019-04-01'}, {'id': None, 'campaign_name': 'CLM - Rentrée été - Cuisine - EN/FR - PO005283', 'reach': '4090', 'clicks': '17', 'spend': '23.24', 'date_start': '2019-04-01', 'date_stop': '2019-04-01'}, {'id

In [7]:
df = pd.DataFrame.from_dict(laSalle_insights)
df = df.assign(Time=pd.to_datetime(df.date_stop)).drop('date_start', axis='columns').drop('date_stop', axis='columns')

# Identify schools and boost
df['campaign_name'] = df['campaign_name'].str.replace("CLM", "laSalle").str.replace("CL", "Boost").str.replace("eLearning", "eLearning")

# Create school column
df['school'] = df["campaign_name"].str.split("-", n=1, expand=True)[0]
df['school'] = df['school'].str.replace(" ", "")

# Intermediate state to get a clean campaign column
df["campaign_rename"] = df["campaign_name"].str.split("-", n=1, expand=True)[1]

# Create PO column
df["po"] = df["campaign_rename"].str.rsplit("-", n=1, expand=True)[1]
df["po"] = df["po"].str.replace(" ", "")

# Create a clean campaign column
df["campaign"] = df["campaign_rename"].str.rsplit("-", n=1, expand=True)[0]

# Get rid of dirty columns
df = df.drop('campaign_name', axis='columns').drop('campaign_rename', axis='columns')

# Reorder columns
clean = df[["campaign", "school", "po", "Time", "spend", "reach","clicks"]]
clean

,campaign,school,po,Time,spend,reach,clicks
0,Campagne permanente - Boost post,Boost,PO004623,2019-04-01,172.24,18251,2282
1,Conversion - Budget Optimization,eLearning,PO000000,2019-04-01,97.93,5898,73
2,Conversion - Discussion DEC Phase 2 - Mars 20...,laSalle,PO005153,2019-04-01,65.08,7588,61
3,Conversion - Discussion DEC Phase 2 - Mars 20...,laSalle,PO005153,2019-04-01,51.4,7744,35
4,Rentrée été - Cuisine - EN/FR,laSalle,PO005283,2019-04-01,23.24,4090,17
5,Rentrée été 2019 - Mtl - FR,laSalle,PO005283,2019-04-01,60.2,4911,50
6,Rentrée été 2019 - Mtl - EN,laSalle,PO005283,2019-04-01,60.52,5161,60
7,Rentrée été 2019 - Laval - FR,laSalle,PO005283,2019-04-01,23.36,1901,23
8,Rentrée été 2019 - Laval - EN,laSalle,PO005283,2019-04-01,15.16,1295,9
9,Rentrée été 2019 - FR,eLearning,PO005154,2019-04-01,106.6,6616,102


In [8]:
# Make sure spend is the right data type
df.spend = df.spend.astype(float)

# Create pivot table to extract some KPI
pivot = pd.pivot_table(df, values='spend', index=['po'], columns=['school'], aggfunc=np.sum, fill_value=0, margins=True, margins_name='total')

# Make a dict of the pivot table
spent = pivot.to_dict()

# Extract the value
boost_spent = spent["Boost"]["total"]
e_learning_spent = spent["eLearning"]["total"]
la_salle_spent = spent["LaSalle"]["total"]

spent

{'Boost': {'PO000000': 0.0,
  'PO004623': 440.43,
  'PO005153': 0.0,
  'PO005154': 0.0,
  'PO005283': 0.0,
  'PO005379': 0.0,
  'total': 440.43},
 'eLearning': {'PO000000': 699.6,
  'PO004623': 0.0,
  'PO005153': 0.0,
  'PO005154': 2848.27,
  'PO005283': 0.0,
  'PO005379': 0.0,
  'total': 3547.87},
 'laSalle': {'PO000000': 0.0,
  'PO004623': 0.0,
  'PO005153': 800.31,
  'PO005154': 0.0,
  'PO005283': 1267.4600000000003,
  'PO005379': 61.8,
  'total': 2129.57},
 'total': {'PO000000': 699.6,
  'PO004623': 440.43,
  'PO005153': 800.31,
  'PO005154': 2848.27,
  'PO005283': 1267.4600000000003,
  'PO005379': 61.8,
  'total': 6117.869999999999}}

In [9]:
df.spend = df.spend.astype(int)
po_pivot = pd.pivot_table(df, values='spend', index=['po'], aggfunc=np.sum, fill_value=0)
po_pivot['spend'] = ": " + po_pivot['spend'].astype(str) + " $"
po_pivot

,spend
po,
PO000000,: 696 $
PO004623,: 439 $
PO005153,: 793 $
PO005154,: 2839 $
PO005283,: 1254 $
PO005379,: 60 $


,spend
po,
PO000000,: 696 $
PO004623,: 439 $
PO005153,: 793 $
PO005154,: 2839 $
PO005283,: 1254 $
PO005379,: 60 $


NameError: name 'PO000000' is not defined

In [6]:
# Get date
today = datetime.today()
tday = today.strftime("%m-%d-%Y")

port = 465  # For SSL
password = os.environ.get("email_pdw")
context = ssl.create_default_context()
sender = os.environ.get("sender")
receiver = os.environ.get("receiver")

message = MIMEMultipart("alternative")
message["Subject"] = "Facebook Report " + str(tday)
message["From"] = sender
message["To"] = receiver

html = """
    <html>

    <head>
      <title> Facebook Report </title>
      <meta http-equiv='Content-Type' content='text/html; charset=UTF-8'>
      <meta name='viewport' content='width=device-width, initial-scale=1'>
      <style type='text/css'>
        # outlook a {
          padding: 0;
        }

        .ReadMsgBody {
          width: 100%;
        }

        .ExternalClass {
          width: 100%;
        }

        .ExternalClass * {
          line-height: 100%;
        }

        body {
          margin: 0;
          padding: 0;
          -webkit-text-size-adjust: 100%;
          -ms-text-size-adjust: 100%;
        }

        table,
        td {
          border-collapse: collapse;
          mso-table-lspace: 0pt;
          mso-table-rspace: 0pt;
        }

        img {
          border: 0;
          height: auto;
          line-height: 100%;
          outline: none;
          text-decoration: none;
          -ms-interpolation-mode: bicubic;
        }

        p {
          display: block;
          margin: 13px 0;
        }
      </style>

      <style type='text/css'>
        @media only screen and (min-width:480px) {
          .mj-column-per-60 {
            width: 60% !important;
            max-width: 60%;
          }

          .mj-column-per-40 {
            width: 40% !important;
            max-width: 40%;
          }

          .mj-column-per-100 {
            width: 100% !important;
            max-width: 100%;
          }

          .mj-column-per-45 {
            width: 45% !important;
            max-width: 45%;
          }

          .mj-column-per-11 {
            width: 11% !important;
            max-width: 11%;
          }

          .mj-column-per-89 {
            width: 89% !important;
            max-width: 89%;
          }

          .mj-column-per-75 {
            width: 75% !important;
            max-width: 75%;
          }

          .mj-column-per-25 {
            width: 25% !important;
            max-width: 25%;
          }

          .mj-column-per-65 {
            width: 65% !important;
            max-width: 65%;
          }

          .mj-column-per-35 {
            width: 35% !important;
            max-width: 35%;
          }
        }
      </style>
      <style type='text/css'>
        @media only screen and (max-width:480px) {
          table.full-width-mobile {
            width: 100% !important;
          }

          td.full-width-mobile {
            width: auto !important;
          }
        }
      </style>
    </head>

    <body>
      <div style=''>
        <div style='Margin:0px auto;max-width:600px;'>
          <table align='center' border='0' cellpadding='0' cellspacing='0' role='presentation' style='width:100%;'>
            <tbody>
              <tr>
                <td style='direction:ltr;font-size:0px;padding:20px 0;text-align:center;vertical-align:top;'>
                  <div class='mj-column-per-60 outlook-group-fix' style='font-size:13px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;'>
                    <table border='0' cellpadding='0' cellspacing='0' role='presentation' style='vertical-align:top;' width='100%'>
                    </table>
                  </div>
                  <div class='mj-column-per-40 outlook-group-fix' style='font-size:13px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;'>
                    <table border='0' cellpadding='0' cellspacing='0' role='presentation' style='vertical-align:top;' width='100%'>
                    </table>
                  </div>
                </td>
              </tr>
            </tbody>
          </table>
        </div>

        <div style='Margin:0px auto;max-width:600px;'>
          <table align='center' border='0' cellpadding='0' cellspacing='0' role='presentation' style='width:100%;'>
            <tbody>
              <tr>
                <td style='direction:ltr;font-size:0px;padding:0px;text-align:center;vertical-align:top;'>

                  <div class='mj-column-per-100 outlook-group-fix' style='font-size:13px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;'>
                    <table border='0' cellpadding='0' cellspacing='0' role='presentation' style='vertical-align:top;' width='100%'>
                      <tr>
                        <td align='center' style='font-size:0px;padding:10px 25px;word-break:break-word;'>
                          <table border='0' cellpadding='0' cellspacing='0' role='presentation' style='border-collapse:collapse;border-spacing:0px;'>
                            <tbody>
                              <tr>
                                <td style='width:550px;'>

                                  <img height='auto' src='https://www.lasallecollege.com/~/media/images/responsive/collegelasalle_montreal/home/visuel-page-accueil-2024x777_v3_en.ashx' style='border:0;display:block;outline:none;text-decoration:none;height:auto;width:100%;'
                                    width='550' />

                                </td>
                              </tr>
                            </tbody>
                          </table>
                        </td>
                      </tr>
                    </table>
                  </div>
                </td>
              </tr>
            </tbody>
          </table>
        </div>
        <div style='Margin:0px auto;max-width:600px;'>
          <table align='center' border='0' cellpadding='0' cellspacing='0' role='presentation' style='width:100%;'>
            <tbody>
              <tr>
                <td style='direction:ltr;font-size:0px;padding:0px;text-align:center;vertical-align:top;'>
                  <div class='mj-column-per-100 outlook-group-fix' style='font-size:13px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;'>
                    <table border='0' cellpadding='0' cellspacing='0' role='presentation' style='vertical-align:top;' width='100%'>
                      <tr>
                        <td align='center' style='font-size:0px;padding:10px 25px;word-break:break-word;'>
                          <div style='font-family:Roboto, Helvetica, sans-serif;font-size:16px;font-weight:300;line-height:24px;text-align:center;color:#616161;'> Your automated report is here! </div>
                        </td>
                      </tr>
                    </table>
                  </div>
                </td>
              </tr>
            </tbody>
          </table>
        </div>
        <div style='Margin:0px auto;max-width:600px;'>
          <table align='center' border='0' cellpadding='0' cellspacing='0' role='presentation' style='width:100%;'>
            <tbody>
              <tr>
                <td style='direction:ltr;font-size:0px;padding:0px;text-align:center;vertical-align:top;'>
                  <div class='mj-column-per-45 outlook-group-fix' style='font-size:13px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;'>
                    <table border='0' cellpadding='0' cellspacing='0' role='presentation' style='vertical-align:top;' width='100%'>
                      <tr>
                        <td align='center' style='font-size:0px;padding:0px;word-break:break-word;'>
                          <div style='font-family:Roboto, Helvetica, sans-serif;font-size:18px;font-weight:500;line-height:24px;text-align:center;color:#616161;'> FACEBOOK REPORT """ + str(tday) + """ </div>
                        </td>
                      </tr>
                      <tr>
                        <td style='font-size:0px;padding:10px 25px;word-break:break-word;'>
                          <p style='border-top:solid 2px #616161;font-size:1;margin:0px auto;width:100%;'> </p>
                        </td>
                      </tr>
                      <tr>
                        <td style='font-size:0px;padding:10px 25px;word-break:break-word;'>
                          <p style='border-top:solid 2px #616161;font-size:1;margin:0px auto;width:45%;'> </p>

                        </td>
                      </tr>
                    </table>
                  </div>

                </td>
              </tr>
            </tbody>
          </table>
        </div>

        <div style='Margin:0px auto;max-width:600px;'>
          <table align='center' border='0' cellpadding='0' cellspacing='0' role='presentation' style='width:100%;'>
            <tbody>
              <tr>
                <td style='direction:ltr;font-size:0px;padding:0px;padding-top:30px;text-align:center;vertical-align:top;'>

                  <div class='mj-column-per-100 outlook-group-fix' style='font-size:13px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;'>
                    <table border='0' cellpadding='0' cellspacing='0' role='presentation' style='vertical-align:top;' width='100%'>
                      <tr>
                        <td align='left' style='font-size:0px;padding:10px 25px;word-break:break-word;'>
                          <div style='font-family:Roboto, Helvetica, sans-serif;font-size:16px;font-weight:300;line-height:24px;text-align:left;color:#616161;'>
                            <p>Hey there!</p>
                            <p> Here are some key metrics from Facebook </p>

                            <p> Last week laSalle spent on Facebook was: """ + str(la_salle_spent) + """$
                              <br>
                              Last week laSalle boost spent on Facebook was: """ + str(boost_spent) + """$
                              <br>
                              Last week eLearning spent on Facebook was: """ + str(e_learning_spent) + """$
                              <br>
                              <br>
                              Spent per PO:
                              """ + tabulate(
        po_pivot, tablefmt='html') + """
                              <br>
                              </p>
                          </div>
                        </td>
                      </tr>
                    </table>
                  </div>

                </td>
              </tr>
            </tbody>
          </table>
        </div>
    """


part1 = MIMEText(html, "html")

message.attach(part1)

with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
    server.login(sender, password)
    server.sendmail(
    sender, receiver, message.as_string()
        )
